In [6]:
from IPython.display import display
import pandas as pd

from simulation.parameters import (
    ASUArrivals, RehabArrivals, ASULOS, RehabLOS,
    ASURouting, RehabRouting, Param
)

def init_param_class(df, unit, parameter, param_class):
    """
    Instantiate a parameter class using values from a DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe with columns "unit", "parameter", "type", "mean" and "sd".
    unit : str
        Unit name to filter by ("asu" or "rehab").
    parameter : str
        Parameter name to filter by ("iat", "los" or "routing").
    param_class: class
        Class to instantiate.

    Returns
    -------
    object
        An instance of param_class initialised with parameters from the
        DataFrame.
    """
    # Filter data to the specified unit and parameter
    df_subset = df[(df["unit"] == unit) & (df["parameter"] == parameter)]

    # If all SD values are missing, create a dict: {type: mean}
    if df_subset["sd"].isnull().all():
        param_dict = df_subset.set_index("type")["mean"].to_dict()
    # Otherwise, create a nested dict with mean and SD for each type
    else:
        param_dict = {}
        for _, row in df_subset.iterrows():
            param_dict[f"{row["type"]}_mean"] = row["mean"]
            param_dict[f"{row["type"]}_sd"] = row["sd"]

    # Instantiate parameter class using dict
    return param_class(**param_dict)



def setup_param_from_csv(csv_path):
    """
    Create a Param instance using parameter values loaded from a CSV file.

    Parameters
    ----------
    csv_path : str
        Path to csv file containing the parameters. Should have columns "unit",
        "parameter", "type", "mean" and "sd". Missing values should be marked
        as "NA".

    Returns
    -------
    Param
        An instance of Param initialised with the parameters from the CSV file.
    """
    # Load parameter data from CSV, treating "NA" as missing values
    df = pd.read_csv(csv_path, na_values=["NA"])

    # Specify mapping of Param() arguments to their corresponding units,
    # parameter types, and parameter classes
    param_specs = [
        ("asu_arrivals", "asu", "iat", ASUArrivals),
        ("rehab_arrivals", "rehab", "iat", RehabArrivals),
        ("asu_los", "asu", "los", ASULOS),
        ("rehab_los", "rehab", "los", RehabLOS),
        ("asu_routing", "asu", "routing", ASURouting),
        ("rehab_routing", "rehab", "routing", RehabRouting),
    ]

    # Instantiate each parameter class and store in a dictionary
    param_kwargs = {
        name: init_param_class(
            df=df, unit=unit, parameter=parameter, param_class=param_class)
        for name, unit, parameter, param_class in param_specs
    }

    # Return a Param instance initialised with all parameter classes
    return Param(**param_kwargs)


display(setup_param_from_csv(csv_path="../data/parametersNM.csv").__dict__)

Initialising parameters...


{'asu_arrivals': RehabArrivals(stroke=0.2, tia=10.3, neuro=100.6, other=1000.2),
 'rehab_arrivals': RehabArrivals(stroke=21.8, neuro=31.7, other=28.6),
 'asu_los': ASULOS(stroke_noesd={'mean': 7.4, 'sd': '8.61'}, stroke_esd={'mean': 4.6, 'sd': '4.8'}, stroke_mortality={'mean': 7.0, 'sd': 8.7}, tia={'mean': 1.8, 'sd': '2.3'}, neuro={'mean': 4.0, 'sd': '5'}, other={'mean': 3.8, 'sd': '5.2'}),
 'rehab_los': RehabLOS(stroke_noesd={'mean': 28.4, 'sd': '27.2'}, stroke_esd={'mean': 30.3, 'sd': '2un3.1'}, tia={'mean': 18.7, 'sd': '23.5'}, neuro={'mean': 27.6, 'sd': '28.4'}, other={'mean': 16.1, 'sd': '14.1'}),
 'asu_routing': ASURouting(stroke={'rehab': 0.24, 'esd': 0.13, 'other': 0.63}, tia={'rehab': 0.01, 'esd': 0.01, 'other': 0.98}, neuro={'rehab': 0.11, 'esd': 0.05, 'other': 0.84}, other={'rehab': 0.05, 'esd': 0.1, 'other': 0.85}),
 'rehab_routing': RehabArrivals(stroke={'esd': 0.4, 'other': 0.6}, neuro={'esd': 0.9, 'other': 0.99}, other={'esd': 0.999, 'other': 0.9999}),
 'warm_up_period':